In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import tiktoken

In [ ]:
GPT_CONFIG_124M = {
 "vocab_size": 50257, # Vocabulary size
 "context_length": 1024, # Context length
 "emb_dim": 768, # Embedding dimension
 "n_heads": 12, # Number of attention heads
 "n_layers": 12, # Number of layers
 "drop_rate": 0.1, # Dropout rate
 "qkv_bias": False # Query-Key-Value bias
}

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads) == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.dropout = nn.Dropout(dropout)
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out)
        self.out_proj = nn.Linear(d_out, d_out)
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(context_length, context_length)))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)
        queries = queries.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2,3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(~mask_bool, -torch.inf)

        attn_weights = torch.softmax(
                attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1,2)
        context_vec = context_vec.contiguous().view(
                b, num_tokens, self.d_out
            )
        context_vec = self.out_proj(context_vec)
        return context_vec

class LayerNorm(nn.Module):
 def __init__(self, emb_dim):
  super().__init__()
  self.eps = 1e-6
  self.scale = nn.Parameter(torch.ones(emb_dim))
  self.shift = nn.Parameter(torch.zeros(emb_dim))

 def forward(self, x):
  mean = x.mean(dim=-1, keepdim=True)
  var = x.var(dim=-1, keepdim=True, unbiased=False)
  norm_x = (x - mean) / torch.sqrt(var + self.eps)
  return self.scale * norm_x + self.shift

class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(
    nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
    nn.GELU(),
    nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
    )
  def forward(self, x):
    return self.layers(x)

class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(
    d_in=cfg["emb_dim"],
    d_out=cfg["emb_dim"],
    context_length=cfg["context_length"],
    num_heads=cfg["n_heads"],
    dropout=cfg["drop_rate"],
    qkv_bias=cfg["qkv_bias"])
    self.ff = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

  def forward(self, x):
    shortcut = x
    x = self.norm1(x)
    x = self.att(x)
    x = self.drop_shortcut(x)
    x = x + shortcut
    shortcut = x
    x = self.norm2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut
    return x

class GPTModel(nn.Module):
 def __init__(self, cfg):
  super().__init__()
  self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
  self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
  self.drop_emb = nn.Dropout(cfg["drop_rate"])

  self.trf_blocks = nn.Sequential(
  *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

  self.final_norm = LayerNorm(cfg["emb_dim"])
  self.out_head = nn.Linear(
  cfg["emb_dim"], cfg["vocab_size"], bias=False
  )
 def forward(self, in_idx):
  batch_size, seq_len = in_idx.shape
  tok_embeds = self.tok_emb(in_idx)

  pos_embeds = self.pos_emb(
  torch.arange(seq_len, device=in_idx.device)
  )
  x = tok_embeds + pos_embeds
  x = self.drop_emb(x)
  x = self.trf_blocks(x)
  x = self.final_norm(x)
  logits = self.out_head(x)
  return logits

In [ ]:
model = GPTModel(GPT_CONFIG_124M)
batch = torch.randint(0, GPT_CONFIG_124M["vocab_size"], (2, GPT_CONFIG_124M["context_length"]))
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[24036, 12745, 32358,  ..., 43544, 31577, 43338],
        [22380, 20155,  2536,  ..., 28892, 37426, 37516]])

Output shape: torch.Size([2, 1024, 50257])
tensor([[[-0.3092,  0.9166, -0.2202,  ..., -0.7472,  1.0774,  0.6277],
         [-0.5404, -0.8863, -0.8809,  ..., -1.0522, -0.6924, -0.6211],
         [ 0.1215, -1.0873,  0.1679,  ...,  0.8339, -0.0861, -0.1788],
         ...,
         [-0.0561,  0.1115, -0.0888,  ...,  0.8872,  0.8018,  0.4042],
         [-0.3731, -0.4267, -0.1247,  ..., -0.4233, -0.2257, -0.8458],
         [ 0.0345, -0.4907,  1.1196,  ...,  0.3589,  0.9235,  0.5893]],

        [[ 0.2318, -0.0465, -0.9149,  ...,  0.3350, -0.0618,  0.9572],
         [ 0.4216, -0.1384, -0.2494,  ..., -1.3333,  0.2891,  0.1393],
         [ 0.3796, -1.2502, -0.3099,  ...,  0.0101, -0.4335, -0.3214],
         ...,
         [ 0.4311,  0.2163, -0.4103,  ...,  0.4634,  0.3200,  0.7896],
         [ 0.1280, -0.2041,  0.3169,  ..., -0.6038, -0.2814, -1.3643],
         [ 0.1

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,018,752


In [ ]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


**Weight tying reduces the overall memory footprint and computational complexity
of the model**

In [ ]:
total_params_gpt2 = (
 total_params - sum(p.numel()
 for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
 f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 124,421,376


In [ ]:
# Memory requirements of the 163 million parameters in our GPTModel object

total_size_bytes = total_params * 4
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.87 MB


# **Generating Texts**

In [ ]:
def generate_text_simple(model, idx,
  max_new_tokens, context_size):
  for _ in range(max_new_tokens):
    idx_cond = idx[:, -context_size:]
    with torch.no_grad():
      logits = model(idx_cond)

    logits = logits[:, -1, :]
    probas = torch.softmax(logits, dim=-1)
    idx_next = torch.argmax(probas, dim=-1, keepdim=True)
    idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0) #Add batch dimension
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


**Put the model into .eval() mode. This disables random components like
dropout, which are only used during training**

In [ ]:
model.eval()
out = generate_text_simple(
 model=model,
 idx=encoded_tensor,
 max_new_tokens=6,
 context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 39986]])
Output length: 5


**Using the .decode method of the tokenizer, we can convert the IDs back into text**

In [ ]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am cached


# Using GPT to generate text

In [ ]:
GPT_CONFIG_124M = {
 "vocab_size": 50257,
 "context_length": 256, #Reduced Context length to 256
 "emb_dim": 768,
 "n_heads": 12,
 "n_layers": 12,
 "drop_rate": 0.1,
 "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (dropout): Dropout(p=0.1, inplace=False)
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (dropout): D